In [1]:
import pandas as pd
from src.spreadsheet import SpreadSheet
from src.classifier import *

/home/rishic/anaconda3/envs/expendo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Constants for the spreadsheet.

In [2]:
SPREADSHEET_ID = "1NONAyZN-DU7VyRR4ystC8rDZ0aBQsEU65kTJiesV-c0"

col_mappings = {
    "month": "I",
    "date": "K",
    "cost": "L",
    "description": "M",
    "category": "N",
    "total": "O"
}

Change this path for each update:

In [3]:
TRANSACTIONS_PATH = "data/data-5-13.csv"

### ETL

Load and preprocess current spreadsheet and new transactions:

In [4]:
sheet = SpreadSheet(SPREADSHEET_ID)
old_data = load_and_process_spreadsheet(sheet, col_mappings)
display(old_data)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=430739421870-r8o3ueiat356loicgp6ntsep34an8pc0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A33489%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=GuF8lSK8Wft1oO1eeqYktEKigH4Qsi&access_type=offline


,date,description,category
1,2024-09-15,Trader Joe's,Groceries
2,2024-09-15,Yama Cafe SF,Meals
3,2024-09-16,Nvidia cafe,Meals
4,2024-09-16,Nvidia cafe,Meals
5,2024-09-16,Nvidia cafe,Meals
...,...,...,...
238,2024-12-30,SQ *ZAREEN'S PALO ALTO Palo Alto CA,Relish
239,2024-12-30,TRADER JOE S #081 MOUNTAIN VIEW CA,Groceries
240,2024-12-30,SAFEWAY #2814 LOS ALTOS CA,Relish
241,2025-01-02,NVIDIA CAFE SANTA CLARA CA,Meals


In [5]:
new_data = load_and_process_transactions(TRANSACTIONS_PATH)
display(new_data)

,Date,Name,Amount
0,2025-01-02,NVIDIA CAFE SANTA CLARA CA,2.50
1,2025-01-02,VOYAGER CAFE SANTA CLARA CA,8.50
2,2025-01-06,TRADER JOE S #081 MOUNTAIN VIEW CA,40.85
3,2025-01-06,PHO TRAN VU MOUNTAIN VIEW CA,54.72
4,2025-01-07,TRADER JOE S #170 MILLBRAE CA,3.49
...,...,...,...
249,2025-05-12,SQ *AUSTIN- HOUNDSTOOT Austin TX,3.00
250,2025-05-12,NVIDIA CAFE SANTA CLARA CA,10.50
251,2025-05-12,NVIDIA CAFE SANTA CLARA CA,2.50
252,2025-05-13,AUST MAD GREENS AUSTIN TX,18.40


Remove overlapping data that has already been added to the sheet:

In [6]:
latest_date = sheet.get_latest_col_value(col_mappings.get("date"))
new_data = new_data[new_data["Date"] >= latest_date]
new_data.rename(columns={"Name": 'description'}, inplace=True)
display(new_data)

,Date,description,Amount
0,2025-01-02,NVIDIA CAFE SANTA CLARA CA,2.50
1,2025-01-02,VOYAGER CAFE SANTA CLARA CA,8.50
2,2025-01-06,TRADER JOE S #081 MOUNTAIN VIEW CA,40.85
3,2025-01-06,PHO TRAN VU MOUNTAIN VIEW CA,54.72
4,2025-01-07,TRADER JOE S #170 MILLBRAE CA,3.49
...,...,...,...
249,2025-05-12,SQ *AUSTIN- HOUNDSTOOT Austin TX,3.00
250,2025-05-12,NVIDIA CAFE SANTA CLARA CA,10.50
251,2025-05-12,NVIDIA CAFE SANTA CLARA CA,2.50
252,2025-05-13,AUST MAD GREENS AUSTIN TX,18.40


### Compute embeddings and classify

In [7]:
threshold = 0.75
new_data_labeled, low_conf = embed_and_classify(old_data, new_data, "description", "category", threshold)

In [8]:
for (index, similarity), (name, label) in low_conf.items():
    print(f"Item '{name}' at index {index} has low confidence ({similarity}); best match was {label}.")

Item 'PHO TRAN VU            MOUNTAIN VIEW CA' at index 3 has low confidence (0.6472660899162292); best match was Meals.
Item 'O2 VALLEY              PALO ALTO     CA' at index 12 has low confidence (0.7046818733215332); best match was Relish.
Item 'ROYAL LAHAINA LODGING  800-2225642   HI' at index 27 has low confidence (0.3652053475379944); best match was Relish.
Item 'GOLDEN BOY PIZZA SF    SAN FRANCISCO CA' at index 28 has low confidence (0.470798134803772); best match was Meals.
Item 'GELATO CLASSICO ITALIA MOUNTAIN VIEW CA' at index 29 has low confidence (0.4500701427459717); best match was Meals.
Item 'TST*MAZRA - RWC        650-503-8440  CA' at index 30 has low confidence (0.5412978529930115); best match was Relish.
Item 'UEP*ZHANGLIANG MALATAN MOUNTAIN VIEW CA' at index 38 has low confidence (0.667631983757019); best match was Meals.
Item 'SULBING CAFE           SANTA CLARA   CA' at index 51 has low confidence (0.6499621868133545); best match was Meals.
Item 'OX9 LANZHOU HANDPU

### Update spreadsheet

In [9]:
display(new_data_labeled)

,Date,description,Amount,category
0,2025-01-02,NVIDIA CAFE SANTA CLARA CA,2.50,Meals
1,2025-01-02,VOYAGER CAFE SANTA CLARA CA,8.50,Meals
2,2025-01-06,TRADER JOE S #081 MOUNTAIN VIEW CA,40.85,Groceries
3,2025-01-06,PHO TRAN VU MOUNTAIN VIEW CA,54.72,None
4,2025-01-07,TRADER JOE S #170 MILLBRAE CA,3.49,Groceries
...,...,...,...,...
249,2025-05-12,SQ *AUSTIN- HOUNDSTOOT Austin TX,3.00,None
250,2025-05-12,NVIDIA CAFE SANTA CLARA CA,10.50,Meals
251,2025-05-12,NVIDIA CAFE SANTA CLARA CA,2.50,Meals
252,2025-05-13,AUST MAD GREENS AUSTIN TX,18.40,None


Reorder and format to match sheet.

In [10]:
import calendar 
new_data_labeled["month"] = pd.to_datetime(new_data_labeled["Date"]).dt.month.apply(lambda x: calendar.month_name[x])
cols = ['month', 'Date', 'Amount', 'description', 'category']
new_data_labeled = new_data_labeled[cols]
new_data_labeled.insert(1, 'blank', '')
display(new_data_labeled)

,month,blank,Date,Amount,description,category
0,January,,2025-01-02,2.50,NVIDIA CAFE SANTA CLARA CA,Meals
1,January,,2025-01-02,8.50,VOYAGER CAFE SANTA CLARA CA,Meals
2,January,,2025-01-06,40.85,TRADER JOE S #081 MOUNTAIN VIEW CA,Groceries
3,January,,2025-01-06,54.72,PHO TRAN VU MOUNTAIN VIEW CA,None
4,January,,2025-01-07,3.49,TRADER JOE S #170 MILLBRAE CA,Groceries
...,...,...,...,...,...,...
249,May,,2025-05-12,3.00,SQ *AUSTIN- HOUNDSTOOT Austin TX,None
250,May,,2025-05-12,10.50,NVIDIA CAFE SANTA CLARA CA,Meals
251,May,,2025-05-12,2.50,NVIDIA CAFE SANTA CLARA CA,Meals
252,May,,2025-05-13,18.40,AUST MAD GREENS AUSTIN TX,None


In [11]:
mo = col_mappings.get("month")
cat = col_mappings.get("category")

next_idx = sheet.get_latest_col_index(mo) + 2
append_range = f"{mo}{next_idx}:{cat}{next_idx}"

Append new data.

In [12]:
sheet.append_values(new_data_labeled.values.tolist(), append_range)

1429 cells appended.


Add stats showing total expenditures in each category for the month.

In [13]:
MONTHS = [2, 3, 4, 5]
for month in MONTHS:
    sheet.add_monthly_stats(month, col_mappings)

Found month entries starting at index 303.
12 cells updated.
Found month entries starting at index 353.
12 cells updated.
Found month entries starting at index 417.
12 cells updated.
Found month entries starting at index 473.
12 cells updated.
